In [41]:
###https://data.mendeley.com/datasets/z8s6w86tr3/2

import pandas as pd
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    confusion_matrix,
    average_precision_score,
)
df = pd.read_csv('../data/Suicide_Detection.csv')

In [42]:
posts = df['text']
labels = df['class'].apply(lambda x: 1 if x == 'suicide' else 0)

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(posts)
y = labels

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
model = LogisticRegression().fit(X_train, y_train)

y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)[:, 1]



In [43]:
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
# roc_auc = roc_auc_score(y_test, y_pred_proba)
# avg_precision = average_precision_score(y_test, y_pred_proba)
conf_matrix = confusion_matrix(y_test, y_pred)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall (Sensitivity/True Positive Rate): {recall:.4f}")
print(f"F1-Score: {f1:.4f}")
# print(f"ROC AUC Score: {roc_auc:.4f}")
# print(f"Average Precision Score (PR AUC): {avg_precision:.4f}")
print("\nConfusion Matrix:")
print(conf_matrix)

Accuracy: 0.9350
Precision: 0.9447
Recall (Sensitivity/True Positive Rate): 0.9241
F1-Score: 0.9343

Confusion Matrix:
[[27457  1569]
 [ 2201 26792]]
